In [1]:
from pykrx import stock
from elasticsearch import Elasticsearch
from konlpy.tag import Mecab

es = Elasticsearch()

# Data

In [23]:
mecab = Mecab()

In [5]:
ticker_ids = stock.get_market_ticker_list("20190225")

tickers = list()
for tk in ticker_ids:
    name = stock.get_market_ticker_name(tk)
    item = {
        'name': name, 
        'ticker': tk, 
        'suggest': {
            'input': mecab.morphs(name), 
            'weight': 1
        }
    }
    tickers.append(item)
    
print('size:', len(tickers))
tickers[:5]

size: 788


[{'name': 'AJ네트웍스',
  'ticker': '095570',
  'suggest': {'input': ['AJ', '네트', '웍스'], 'weight': 1}},
 {'name': 'SK렌터카',
  'ticker': '068400',
  'suggest': {'input': ['SK', '렌터카'], 'weight': 1}},
 {'name': 'AK홀딩스',
  'ticker': '006840',
  'suggest': {'input': ['AK', '홀딩스'], 'weight': 1}},
 {'name': 'BGF',
  'ticker': '027410',
  'suggest': {'input': ['BGF'], 'weight': 1}},
 {'name': 'BGF리테일',
  'ticker': '282330',
  'suggest': {'input': ['BGF', '리', '테일'], 'weight': 1}}]

# Insert 

 - Index: Database 
 - Datatype: Document Type
 - Id: Document ID

In [6]:
# Delete ticker index
es.indices.delete(index='tickers', ignore=[400, 404])

# Mapping
mappings = {
    'mappings': {
        'properties': {
            'name': {'type': 'text'}, 
            'ticker': {'type': 'text'}, 
            'suggest': {'type': 'completion'}
        }
    }
}
es.indices.create('tickers', body=mappings)


# Insert tickers
[es.index(index='tickers', body=tk) for tk in tickers]

[{'_index': 'tickers',
  '_type': '_doc',
  '_id': 'p17LRnYByPpEVaaD3rcv',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 0,
  '_primary_term': 1},
 {'_index': 'tickers',
  '_type': '_doc',
  '_id': 'qF7LRnYByPpEVaaD3rdE',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 1,
  '_primary_term': 1},
 {'_index': 'tickers',
  '_type': '_doc',
  '_id': 'qV7LRnYByPpEVaaD3rdS',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 2,
  '_primary_term': 1},
 {'_index': 'tickers',
  '_type': '_doc',
  '_id': 'ql7LRnYByPpEVaaD3rdf',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 3,
  '_primary_term': 1},
 {'_index': 'tickers',
  '_type': '_doc',
  '_id': 'q17LRnYByPpEVaaD3rdt',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1

# Search

## Match Query

완전 일치해야 합니다.<br>
예를 들어 '000660'은 검색이 되지만 '00066'은 앞글자가 맞더라도 검색이 안됩니다.

In [7]:
query = {
    'query': {
        'match': {
            'ticker': '000660'
        }
    }
}
es.search(index='tickers', body=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 6.2653008,
  'hits': [{'_index': 'tickers',
    '_type': '_doc',
    '_id': 'DV7LRnYByPpEVaaD47iu',
    '_score': 6.2653008,
    '_source': {'name': 'SK하이닉스',
     'ticker': '000660',
     'suggest': {'input': ['SK', '하이닉스'], 'weight': 1}}}]}}

## Match ALL

In [30]:
es.search(index='tickers', 
          body={'query': {'match_all': {}}}, 
          size=1000)

{'took': 24,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 788, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'tickers',
    '_type': '_doc',
    '_id': '417LRnYByPpEVaaD_bka',
    '_score': 1.0,
    '_source': {'name': '지투알',
     'ticker': '035000',
     'suggest': {'input': ['지투알'], 'weight': 1}}},
   {'_index': 'tickers',
    '_type': '_doc',
    '_id': '5F7LRnYByPpEVaaD_bkp',
    '_score': 1.0,
    '_source': {'name': '진도',
     'ticker': '088790',
     'suggest': {'input': ['진도'], 'weight': 1}}},
   {'_index': 'tickers',
    '_type': '_doc',
    '_id': '5V7LRnYByPpEVaaD_bk4',
    '_score': 1.0,
    '_source': {'name': '진양산업',
     'ticker': '003780',
     'suggest': {'input': ['진양', '산업'], 'weight': 1}}},
   {'_index': 'tickers',
    '_type': '_doc',
    '_id': '5l7LRnYByPpEVaaD_blE',
    '_score': 1.0,
    '_source': {'name': '진양폴리',
     'ticker': '010640',
     'suggest': {'input': ['진양', 

## Prefix Query

앞글자가 똑같아야 합니다. 예를 들어 "하이닉스" 로는 검색이 안됩니다.

In [17]:
query = {
    'query': {
        'prefix': {
            'name': {
                'value': 'ㅎ'
            }
        }
    }
}
es.search(index='tickers', body=query)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

## Edge N-gram

In [9]:
ret = es.indices.analyze({'tokenizer': 'edge_ngram',
                          'text': ['SK', '하이닉스']})

[i['token'] for i in ret['tokens']]

['S', 'SK', '하', '하이']

## Suggest

In [15]:
query = {
    "suggest": {
        "stock-suggest": {
            'prefix': '에',
            'completion': {
                'field': 'suggest'
            }}}}

es.search(index='tickers', body=query)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'suggest': {'stock-suggest': [{'text': '에',
    'offset': 0,
    'length': 1,
    'options': [{'text': '에',
      '_index': 'tickers',
      '_type': '_doc',
      '_id': 'Gl7MRnYByPpEVaaDALon',
      '_score': 1.0,
      '_source': {'name': '티에이치엔',
       'ticker': '019180',
       'suggest': {'input': ['티', '에', '이치엔'], 'weight': 1}}},
     {'text': '에',
      '_index': 'tickers',
      '_type': '_doc',
      '_id': 'cV7LRnYByPpEVaaD97kF',
      '_score': 1.0,
      '_source': {'name': '에쓰씨엔지니어링',
       'ticker': '023960',
       'suggest': {'input': ['에', '쓰', '씨', '엔지니어링'], 'weight': 1}}},
     {'text': '에',
      '_index': 'tickers',
      '_type': '_doc',
      '_id': 'dl7LRnYByPpEVaaD97lI',
      '_score': 1.0,
      '_source': {'name': '에이프로젠 KIC',
       'ticker': '007460',
       'sugge